<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("BCEAO.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,country,label,indexed_at,provider_code,series_code,series_name,series_num
0,1960,1960-01-01,25.8,annual,TOFE,TABLEAU DES OPERATIONS FINANCIERES DE L'ETAT (...,AAA,FP1003A0AP,2025-03-19T01:29:40.104Z,BCEAO,AAAFP1003A0AP,COTE D'IVOIRE – RECETTES COURANTES (R3),Appended dataset 2
1,1960,1960-01-01,NA,annual,TOFE,TABLEAU DES OPERATIONS FINANCIERES DE L'ETAT (...,AAA,FP1061A0AP,2025-03-19T01:29:40.104Z,BCEAO,AAAFP1061A0AP,COTE D'IVOIRE – Financement exterieur par vari...,Appended dataset 34
2,1960,1960-01-01,NA,annual,PIBC,PIB ET SES EMPLOIS A PRIX CONSTANT Base 100 =2...,AAA,SR1022A0BQ,2025-03-19T01:29:40.104Z,BCEAO,AAASR1022A0BQ,COTE D'IVOIRE – VARIATION DE STOCKS,Appended dataset 8
3,1960,1960-01-01,NA,annual,SIM,SITUATION DES INSTITUTIONS MONETAIRES (Million...,AAA,SF1256A0AP,2025-01-18T01:18:08.259Z,BCEAO,AAASF1256A0AP,COTE D'IVOIRE – - Banques,Appended dataset 3
4,1960,1960-01-01,NA,annual,DPE,DETTE PUBLIQUE EXTERIEURE,AAA,FP3005A0FA,2025-03-21T01:13:04.470Z,BCEAO,AAAFP3005A0FA,COTE D'IVOIRE – Dette envers la Banque mondiale,Appended dataset 3


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("BCEAO.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,country,label,Country,Label
0,annual,BCEAO,PIBN,PIB ET SES EMPLOIS EN VALEUR NOMINALE (en mill...,AAASR1011A0BP,COTE D'IVOIRE – - Impots indirects,1960,1960-01-01,8.7,8.7,AAA,SR1011A0BP,COTE D'IVOIRE,- Impots indirects
65,annual,BCEAO,PIBN,PIB ET SES EMPLOIS EN VALEUR NOMINALE (en mill...,AAASR1012A0BP,COTE D'IVOIRE – PIB marchand,1960,1960-01-01,124.5,124.5,AAA,SR1012A0BP,COTE D'IVOIRE,PIB marchand
130,annual,BCEAO,PIBN,PIB ET SES EMPLOIS EN VALEUR NOMINALE (en mill...,AAASR1013A0BP,COTE D'IVOIRE – PIB non marchand,1960,1960-01-01,16.2,16.2,AAA,SR1013A0BP,COTE D'IVOIRE,PIB non marchand
195,annual,BCEAO,PIBN,PIB ET SES EMPLOIS EN VALEUR NOMINALE (en mill...,AAASR1015A0BP,COTE D'IVOIRE – PIB nominal (en milliards de F...,1960,1960-01-01,140.7,140.7,AAA,SR1015A0BP,COTE D'IVOIRE,PIB nominal (en milliards de FCFA)
260,annual,BCEAO,PIBN,PIB ET SES EMPLOIS EN VALEUR NOMINALE (en mill...,AAASR1016A0BP,COTE D'IVOIRE – CONSOMMATION FINALE,1960,1960-01-01,109.4,109.4,AAA,SR1016A0BP,COTE D'IVOIRE,CONSOMMATION FINALE


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

False

<font size=5, font color='blue'> The provider reused series codes in several datasets, so we now need ['period_start_day', 'series_code', 'dataset_code'] to identify a unique row for stata_df and python_df

In [6]:
sum(stata_df.duplicated(subset=cols+['dataset_code']))+sum(python_df.duplicated(subset=cols+['dataset_code'])) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code', 'dataset_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','dataset_code', 'value']] sliced from stata_df and python_df are exactly the same.

In [7]:
stata_sub = stata_df.sort_values(by=cols+['dataset_code'])[cols+['dataset_code', 'value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols+['dataset_code'])[cols+['dataset_code', 'value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True